# Importation des librairies

In [218]:
import pandas as pd
import numpy as np
import requests
import cartiflette.s3 as s3
import geopandas as gpd
from shapely.geometry import shape
from io import StringIO
from sklearn.impute import KNNImputer

# Importation des bases

## Import des bases de données via API de l'OpenDataSoft

L'éxécution de la requête API prend assez de temps

In [219]:
def get_data_ODS(dataset_name):
    """
    Fonction qui permet de récupérer des données via l'API d'OpenDataSoft
    Elle retourne un dataframe
    
    """
    df = pd.DataFrame()
    # URL de base pour accéder à l'API d'ODS
    base_url_ODS = "https://odre.opendatasoft.com/api/explore/v2.1"
    
    dataset_path = f"/catalog/datasets/{dataset_name}/exports/json?lang=fr&timezone=Europe%2FBerlin"
    url = f"{base_url_ODS}{dataset_path}"
    
    # Exécution de la requête GET
    
    response = requests.get(url)
    # Vérification si la requête a réussi
    if response.status_code == 200:
        # Extraction des données
        data = response.json()
        # Conversion des résultats en DataFrame
        df = pd.DataFrame(data)
    else:
        print(f"Erreur lors de la requête: {response.status_code}")

    return df


def get_data_ODSW(dataset_name, output = "json"):
    """
    Fonction qui permet de récupérer des données via l'API  publique d'OpenDataSoft
    Elle retourne un dataframe
    
    """
    df = pd.DataFrame()
    # URL de base pour accéder à l'API d'ODS
    base_url_ODS = "https://public.opendatasoft.com/api/explore/v2.1"
    
    dataset_path = f"/catalog/datasets/{dataset_name}/exports/{output}?lang=fr&timezone=Europe%2FBerlin"
    url = f"{base_url_ODS}{dataset_path}"
    
    # Exécution de la requête GET
    response = requests.get(url)
    # Vérification si la requête a réussi
    if response.status_code == 200:
        # Extraction des données
        if output == "json":
            data = response.json()
            # Conversion des résultats en DataFrame
            df = pd.DataFrame(data)
        else:
            data = response.json()
            geometries = [shape(feature['geometry']) for feature in data['features']]
            properties = [feature['properties'] for feature in data['features']]
    
            # Créer un GeoDataFrame en combinant les géométries et les propriétés
            df = gpd.GeoDataFrame(geometry=geometries, data=properties)
    else:
        print(f"Erreur lors de la requête: {response.status_code}")

    return df

In [4]:
consumption_dataset_name = "consommation-quotidienne-brute-regionale"
weather_dataset_name = "donnees-synop-essentielles-omm"
#geographic_dataset_name = "georef-france-commune"
holiday_dataset_name = "jours-ouvres-week-end-feries-france-2010-a-2030"
data_consumption = get_data_ODS(consumption_dataset_name)
data_weather = get_data_ODSW(weather_dataset_name)
#data_geo = get_data_ODSW(geographic_dataset_name, output="geojson")
holiday_dataset = get_data_ODSW(holiday_dataset_name)

On traite les colonnes du dataset geographique pour enregistrer les données sous format GeoJson

In [31]:
#On recupère les colonnes importantes de la base de données
#geo_cols = ['geo_point_2d', 'year', 'reg_code', 'reg_name', 'dep_code', 'dep_name','geometry']
#data_geo = data_geo[geo_cols]
#treated_geo_cols = ['year', 'reg_code', 'reg_name', 'dep_code', 'dep_name']
##On effectue un nettoyage des données contenues dans les colonnes
#data_geo[treated_geo_cols] = data_geo[treated_geo_cols].applymap(lambda x: x.replace('[', '').replace(']', '') if isinstance(x, str) else x)
#data_geo[treated_geo_cols] = data_geo[treated_geo_cols].applymap(lambda x: ''.join(filter(lambda char: char.isalnum() or char in ['_', '-', '@', '#'], str(x))))

,geo_point_2d,year,reg_code,reg_name,dep_code,dep_name,geometry
0,"{'lon': 4.186473859960533, 'lat': 48.520816933...",2023,44,GrandEst,10,Aube,"POLYGON ((4.18730 48.54392, 4.18724 48.54250, ..."
1,"{'lon': 4.598596436015939, 'lat': 48.306537350...",2023,44,GrandEst,10,Aube,"POLYGON ((4.62567 48.32423, 4.62429 48.32356, ..."
2,"{'lon': 4.6153344590215, 'lat': 48.44294278327...",2023,44,GrandEst,10,Aube,"POLYGON ((4.63562 48.45660, 4.63535 48.45667, ..."
3,"{'lon': 3.8349133102621433, 'lat': 48.10864367...",2023,44,GrandEst,10,Aube,"POLYGON ((3.80166 48.10673, 3.80124 48.10597, ..."
4,"{'lon': 1.8228167609346886, 'lat': 43.33065948...",2023,76,Occitanie,11,Aude,"POLYGON ((1.83985 43.32973, 1.83966 43.32982, ..."


## Import d'une base de données complémentaire (celle des régions, communes, et départements de France

Cette importation va se faire à travers une requête via l'API de Datagouv

In [220]:
#Identifiant du dataset
dataset_id = "4a7c837bb6da8e363604082bcc8b2e504cf08038"
dataset_sha1 = "dbe8a621-a9c4-4bc3-9cae-be1699c5ff25"

# URL de base pour accéder à l'API
base_url = "https://www.data.gouv.fr/api/1/"

# Chemin pour accéder aux enregistrements du dataset
dataset_path = f"datasets/r/{dataset_sha1}?dataset={dataset_id}"

# Construction de l'URL complète
url = f"{base_url}{dataset_path}"

# Exécution de la requête GET
response = requests.get(url)

# Vérification si la requête a réussi
if response.status_code == 200:
    # Lecture du contenu CSV
    data_communes = pd.read_csv(StringIO(response.content.decode('utf-8')))
else:
    print(f"Erreur lors de la requête: {response.status_code}")

On enregistre la base pour ne plus avoir à éxécuter la requête API à chaque étape

In [6]:
data_consumption.to_csv('data_consumption.csv', index=False)
data_communes.to_csv('data_communes.csv', index=False)
data_weather.to_csv('data_weather.csv', index = False)
#data_geo.to_file("data_geo.geojson", driver='GeoJSON')

In [221]:
data_consumption = pd.read_csv('data_consumption.csv')
data_communes = pd.read_csv('data_communes.csv')
data_weather = pd.read_csv('data_weather.csv')
#data_geo = gpd.read_file("data_geo.geojson")

In [222]:
dep = s3.download_vectorfile_url_all(
    values = "metropole",
    crs = 4326,
    borders = "DEPARTEMENT",
    vectorfile_format="topojson",
    filter_by="FRANCE_ENTIERE",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=DEPARTEMENT/crs=4326/FRANCE_ENTIERE=metropole/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


Downloading: : 7.17MiB [00:00, 83.3MiB/s]


Les dataframes de consommation et météorologiques  étant sous forme de séries temporelles, on place les dates en index.

In [223]:
data_consumption['date'] = pd.to_datetime(data_consumption['date'])
data_consumption.set_index('date', inplace=True)

In [224]:
data_consumption.head()

,date_heure,heure,code_insee_region,region,consommation_brute_gaz_grtgaz,statut_grtgaz,consommation_brute_gaz_terega,statut_terega,consommation_brute_gaz_totale,consommation_brute_electricite_rte,statut_rte,consommation_brute_totale
date,,,,,,,,,,,,
2019-02-01,2019-02-01T06:00:00+01:00,06:00,27,Bourgogne-Franche-Comté,5053.0,Définitif,NaN,NaN,5053.0,3052,Définitif,8105.0
2019-02-01,2019-02-01T07:00:00+01:00,07:00,24,Centre-Val de Loire,4079.0,Définitif,NaN,NaN,4079.0,3117,Définitif,7196.0
2019-02-01,2019-02-01T07:00:00+01:00,07:00,52,Pays de la Loire,5795.0,Définitif,NaN,NaN,5795.0,4646,Définitif,10441.0
2019-02-01,2019-02-01T07:30:00+01:00,07:30,24,Centre-Val de Loire,NaN,NaN,NaN,NaN,NaN,3252,Définitif,NaN
2019-02-01,2019-02-01T07:30:00+01:00,07:30,28,Normandie,NaN,NaN,NaN,NaN,NaN,4911,Définitif,NaN


Choix de l'année

In [225]:
annee_choisie = 2019
data_consumption = data_consumption[data_consumption.index.year == annee_choisie]


In [227]:
col_del = pd.Index(['date_heure', 'consommation_brute_gaz_grtgaz', 'statut_grtgaz', 'consommation_brute_gaz_terega', 'statut_terega','consommation_brute_gaz_totale','statut_rte', 'consommation_brute_totale'])
data_consumption.drop(col_del, axis=1,inplace=True)
data_consumption.isna().sum()

heure                                 0
code_insee_region                     0
region                                0
consommation_brute_electricite_rte    0
dtype: int64

## Aggrégation des consommations par date et par région

In [228]:
data_consumption1 = data_consumption.groupby(['code_insee_region','date'], as_index = False)
data_consumption2 = data_consumption.groupby(['code_insee_region','date','heure'], as_index = False)

In [229]:
data_consumption1 = data_consumption1['consommation_brute_electricite_rte'].sum()
data_consumption2 = data_consumption2['consommation_brute_electricite_rte'].sum()

In [230]:
data_consumption1.head()

,code_insee_region,consommation_brute_electricite_rte
0,11,345462
1,11,356965
2,11,414973
3,11,429347
4,11,484779


### Calcul des superficies par département

### Calcul de la superficie

In [231]:
## On divise par par 1e6 pour obtenir la superficie par commune en km²
#data_geo['superficie_comm'] = data_geo['geometry'].area / 1e6  
#
##On fait un regroupement par département, et on calcule la superficie par département en sommant les autres superficies
#data_geo['superficie_dep'] = data_geo.groupby('dep_code')['superficie_comm'].transform('sum')
dep["superficie"] = dep.to_crs(2154).area

### Traitement de la base weather

In [232]:
nouveaux_noms_colonnes={'numer_sta':'num_station', 'date':'date_UTC', 'pmer':'pression_mer', 'tend':'variation_pression_3h', 'cod_tend':'type_tendance_baro', 'dd':'direction_vent', 'ff':'vitesse_vent', 't':'temperature', 'td':'point_de_rosee',
       'u':'humidite', 'vv':'visibilite_horizontale', 'ww':'temps_present', 'w1':'temps_passe_1', 'w2':'temps_passe_2', 'n':'nebulosite_totale', 'nbas':'nebulosite_nuage_etage_inf', 'hbas':'hauteur_base_nuage_etage_inf', 'cl':'type_nuage_etage_inf', 'cm':'type_nuage_etage_moyen', 'ch':'type_nuage_etage_sup',
       'pres':'pression_station', 'niv_bar':'niveau_barometriq', 'geop':'geopotentiel', 'tend24':'variation_pression_24h', 'tn12':'temperature_min_sur_12h', 'tn24':'temperature_min_sur_24h', 'tx12':'temperature_max_sur_12h', 'tx24':'temperature_max_sur_24h',
       'tminsol':'temperature_min_sol_sur_12h', 'sw':'methode_mesure_temperature_thermometre_mouille', 'tw':'temperature_thermometre_mouille', 'raf10':'rafales_10_dernieres_minutes', 'rafper':'rafales_sur_une_periode', 'per':'periode_mesure_rafales', 'etat_sol':'etat_du_sol', 'ht_neige':'hauteur_totale_couche_neige/glace_au_sol',
       'ssfrai':'hauteur_neige_fraiche', 'perssfrai':'periode_mesure_neige_fraiche', 'rr1':'precipitation_dans_1_derniere_heure', 'rr3':'precipitation_dans_3_derniere_heure', 'rr6':'precipitation_dans_6_derniere_heure', 'rr12':'precipitation_dans_12_derniere_heure', 'rr24':'precipitation_dans_24_derniere_heure', 'phenspe1':'phenomene_special_1',
       'phenspe2':'phenomene_special_2', 'phenspe3':'phenomene_special_3', 'phenspe4':'phenomene_special_4', 'nnuage1':'nebulosite_couche_nuageuse_1', 'ctype1':'type_de_nuage_1', 'hnuage1':'hauteur_de_base_nuage_1',
       'nnuage2':'nebulosite_couche_nuageuse_2', 'ctype2':'type_de_nuage_2', 'hnuage2':'hauteur_de_base_nuage_2', 'nnuage3':'nebulosite_couche_nuageuse_3', 'ctype3':'type_de_nuage_3', 'hnuage3':'hauteur_de_base_nuage_3',
       'nnuage4':'nebulosite_couche_nuageuse_4', 'ctype4':'type_de_nuage_4', 'hnuage4':'hauteur_de_base_nuage_4', 'coordonnees':'coordonnees', 'nom':'nom_commune',
       'type_de_tendance_barometrique':'type_de_tendance_barometrique', 'temps_passe_1':'temps_passe_1', 'temps_present':'temps_present', 'tc':'temperature_degre_celcius',
       'tn12c':'temperature_celcius_min_sur_12h', 'tn24c':'temperature_celcius_min_sur_24h', 'tx12c':'temperature_celcius_max_sur_12h', 'tx24c':'temperature_celcius_max_sur_24h', 'tminsolc':'temperature_celcius_min_sol_sur_12h', 'latitude':'latitude', 'longitude':'longitude',
       'altitude':'altitude', 'libgeo':'libelle_geolocalisation', 'codegeo':'code_geolocalisation', 'nom_epci':'nom_EPCI', 'code_epci':'code_EPCI', 'nom_dept':'nom_departement',
       'code_dep':'code_departement', 'nom_reg':'nom_region', 'code_reg':'code_region', 'mois_de_l_annee':'mois'}

In [492]:
base_meteo_modifie_1 = data_weather.rename(columns=nouveaux_noms_colonnes)

## Identification des variables météoroliques à retenir

Il est question d'identifier les les variables météorologiques se reférant aux espects retenus Après une revue de la littérature:
* __Température :__ les variables identifiées sont la température en degré celcius ('temperature_degre_celcius'), la température minimale sur les 24 dernières heures ('temperature_celcius_min_sur_24h'), et la température maximale sur les 24 dernières heures ('temperature_celcius_max_sur_24h')
* __Vitesse du vent :__ la variable identifiée est la vitesse du vent ('vitesse_vent')
* __Couverture nuageuse :__ la variable identifié est la nébulosité totale et renvoie à la quantité totale de nuages couvrant le ciel ou plus simplement le degré de couverture nuageuse dans le ciel ('nebulosite_totale')
* __Humidité :__ la variable identifiée est l'indice d'humidité ('humidite')
* __Rayonnement global :__ la variable identifiée pour capter cet aspect est la distance maximale à laquelle des objets peuvent être clairement discernés à l'horizon ('visibilite_horizontale')

In [493]:
weather_vars = ['num_station', 'date_UTC', 'vitesse_vent', 'temperature','nebulosite_totale','temperature_degre_celcius', 'temperature_celcius_min_sur_24h', 'temperature_celcius_max_sur_24h','latitude', 'longitude','altitude', 'libelle_geolocalisation', 'code_geolocalisation','nom_departement', 'code_departement','nom_region', 'code_region', 'mois', 'visibilite_horizontale', 'coordonnees','nom_commune',]
base_meteo_modifie_1 = base_meteo_modifie_1[weather_vars]

### Vérification et gestion des valeurs manquantes pour les variables d'agrégation

#### Vérification des régions et départements manquants

In [494]:
print("Nombre de nom_regions manquants", base_meteo_modifie_1['nom_region'].isnull().sum())
print("Nombre de code_regions manquants", base_meteo_modifie_1['code_region'].isnull().sum())
print("Nombre de nom_departements manquants", base_meteo_modifie_1['nom_departement'].isnull().sum())
print("Nombre decode_departements  manquants", base_meteo_modifie_1['code_departement'].isnull().sum())
print("Nombre de nom_communes manquants", base_meteo_modifie_1['nom_commune'].isnull().sum())

Nombre de nom_regions manquants 242657
Nombre de code_regions manquants 242657
Nombre de nom_departements manquants 242657
Nombre decode_departements  manquants 242657
Nombre de nom_communes manquants 0


On constate que tous les noms de commune sont renseignés, mais certains noms de région et département ne le sont pas. Nous allons donc utiliser les noms de commune pour renseigner les départements et régions manquants.

Nous explorons la possibilité de completer les régions et départements manquants à partir de la base de données des communes, régions et départements.

In [495]:
data_communes.isna().sum()

code_commune_INSEE          0
nom_commune_postal          0
code_postal                 0
libelle_acheminement        0
ligne_5                 33384
latitude                   91
longitude                  91
code_commune                3
article                 31419
nom_commune                 0
nom_commune_complet         0
code_departement            0
nom_departement            89
code_region                 0
nom_region                 89
dtype: int64

Le dataset des communes possède lui même des valeurs manquantes au niveau des codes de département et de région. Nous allons compléter ces valeurs manquantes en utilisant les combinaisons "code_commune - code_departement - code-region" les plus fréquentes. Puis nous allons le joindre avec le dataset de météo.

In [496]:
combinaisons_frequentes = data_communes.groupby(['code_commune', 'code_departement', 'code_region']).size().idxmax()

# Imputer les valeurs manquantes dans 'longitude' et 'latitude' en fonction des combinaisons les plus fréquentes
data_communes['code_departement'].fillna(combinaisons_frequentes[1], inplace=True)
data_communes['code_region'].fillna(combinaisons_frequentes[2], inplace=True)

In [497]:
combi_freq = base_meteo_modifie_1.groupby(['latitude', 'longitude', 'libelle_geolocalisation']).size().idxmax()

# Imputer les valeurs manquantes dans 'longitude' et 'latitude' en fonction des combinaisons les plus fréquentes
base_meteo_modifie_1['libelle_geolocalisation'].fillna(combi_freq[2], inplace=True)

Cela étant fait, comme le dataset weather possède les noms de communes (la colonne "nom_commune"), qui est déjà presqu'harmonisée avec la colonne des noms de communes du dataset de communes (nom_commune_complet). On complète alors l'hamonisation et on peut donc faire une fusion sur les noms de communes.

Harmonisation

In [498]:
base_meteo_modifie_1.isna().sum()

num_station                              0
date_UTC                                 0
vitesse_vent                         26924
temperature                          15274
nebulosite_totale                  1203829
temperature_degre_celcius            15274
temperature_celcius_min_sur_24h    2143006
temperature_celcius_max_sur_24h    2143902
latitude                                 0
longitude                                0
altitude                                 0
libelle_geolocalisation                  0
code_geolocalisation                242657
nom_departement                     242657
code_departement                    242657
nom_region                          242657
code_region                         242657
mois                                     0
visibilite_horizontale              547882
coordonnees                              0
nom_commune                              0
dtype: int64

In [499]:
base_meteo_modifie_1['libelle_geolocalisation'] = base_meteo_modifie_1['libelle_geolocalisation'].apply(lambda x: x.replace("Saint-Pierre", "St pierre").replace("Saint-Barthélemy", "St barthelemy") if isinstance(x, str) else x)

In [500]:
#Vérifions que les noms sont bien harmonisés
a = pd.DataFrame()
for elt in list(base_meteo_modifie_1['libelle_geolocalisation'].unique()):
    if "cod_geo" in a.columns:
        if not data_communes[data_communes['nom_commune_complet']==elt].empty:
            nvo = {"lib_geo" : elt, "cod_geo" : base_meteo_modifie_1[base_meteo_modifie_1['libelle_geolocalisation'] == elt ]['code_geolocalisation'].iloc[0], "nom_com" :data_communes[data_communes['nom_commune_complet']==elt]["nom_commune_complet"].iloc[0], "cod_com" : data_communes[data_communes['nom_commune_complet']==elt]["code_commune_INSEE"].iloc[0], "nom_dep": data_communes[data_communes['nom_commune_complet']==elt]["nom_departement"].iloc[0], "nom_reg" : data_communes[data_communes['nom_commune_complet']==elt]["nom_region"].iloc[0]}
            a = pd.concat([a, pd.DataFrame([nvo])], ignore_index=True)
    else:
        a["lib_geo"] = elt
        a["cod_geo"] = base_meteo_modifie_1[base_meteo_modifie_1['libelle_geolocalisation'] == elt ]['code_geolocalisation'][0]
        if not data_communes[data_communes['code_commune_INSEE']==elt].empty:
            a["nom_com"] = data_communes[data_communes['nom_commune_complet']==elt]["nom_commune_postal"].iloc[0]
            a["cod_com"] = data_communes[data_communes['nom_commune_complet']==elt]["code_commune_INSEE"].iloc[0]
            a["nom_dep"] = data_communes[data_communes['nom_commune_complet']==elt]["nom_departement"].iloc[0]
            a["nom_reg"] = data_communes[data_communes['nom_commune_complet']==elt]["nom_region"].iloc[0]

a['resultat_comparaison'] = a['lib_geo'] == a['nom_com']
a

,lib_geo,cod_geo,nom_com,cod_com,nom_dep,nom_reg,resultat_comparaison
0,Marignane,NaN,Marignane,13054,Bouches-du-Rhône,Provence-Alpes-Côte d'Azur,True
1,Carpiquet,14137,Carpiquet,14137,Calvados,Normandie,True
2,Guipavas,29075,Guipavas,29075,Finistère,Bretagne,True
3,Embrun,05046,Embrun,5046,Hautes-Alpes,Provence-Alpes-Côte d'Azur,True
4,Le Lamentin,97213,Le Lamentin,97213,Martinique,Martinique,True
5,Fretin,59256,Fretin,59256,Nord,Hauts-de-France,True
6,Athis-Mons,91027,Athis-Mons,91027,Essonne,Île-de-France,True
7,Saint-Georges,97308,Saint-Georges,15188,Cantal,Auvergne-Rhône-Alpes,True
8,La Hague,50041,La Hague,50041,Manche,Normandie,True
9,Ajaccio,2a004,Ajaccio,2A004,Corse-du-Sud,Corse,True


### Jointure des bases communes et weather

In [501]:
data_communes = data_communes.drop_duplicates(subset='nom_commune_complet', keep='first')

In [502]:
base_meteo_modifie_1 = pd.merge(base_meteo_modifie_1, data_communes, left_on='libelle_geolocalisation', right_on="nom_commune_complet",  how='left', validate="m:1")

In [503]:
print("Nombre de valeurs uniques de combinaisons latitude - longitude : ", base_meteo_modifie_1.groupby(['latitude_x', 'longitude_x']).size().nunique())
print("Nombre de régions couvertes : ", base_meteo_modifie_1['nom_region_x'].nunique())
print("Nombre de communes renseignées dans la base: ", base_meteo_modifie_1['nom_commune_x'].nunique())

Nombre de valeurs uniques de combinaisons latitude - longitude :  61
Nombre de régions couvertes :  21
Nombre de communes renseignées dans la base:  62


Il semble dès lors que la base a couvert toutes les régions de France, mais avec uniquement 61 points de collecte d'informations (à travers toute la France), correspondants à 62 communes. Comme toutes les latitudes et longitudes sont renseignées, à chaque couple (latitude - longitude) correspond un point de collecte, donc une commune, une région et un département. On va donc imputer tout d'abord les communes en fonction de ce qu'il a rencontré sur une autre ligne correspondant à la séquence longitude - latitude - commune, puis imputer département et région par la suite.

In [504]:
# Tri du DataFrame par 'longitude' et 'latitude'
base_meteo_modifie_1.sort_values(['longitude_x', 'latitude_x'], inplace=True)

# Remplir les valeurs manquantes dans 'departement' et région en fonction des combinaisons 'longitude-latitude'
base_meteo_modifie_1['code_departement_x'] = base_meteo_modifie_1.groupby(['longitude_x', 'latitude_x'])['code_departement_x'].transform(lambda x: x.ffill().bfill())
base_meteo_modifie_1['code_region_x'] = base_meteo_modifie_1.groupby(['longitude_x', 'latitude_x'])['code_region_x'].transform(lambda x: x.ffill().bfill())

In [505]:
base_meteo_modifie_1['code_departement_y'] = base_meteo_modifie_1.groupby(['longitude_y', 'latitude_y'])['code_departement_y'].transform(lambda x: x.ffill().bfill())
base_meteo_modifie_1['code_region_y'] = base_meteo_modifie_1.groupby(['longitude_y', 'latitude_y'])['code_region_y'].transform(lambda x: x.ffill().bfill())

In [506]:
b = base_meteo_modifie_1[base_meteo_modifie_1['code_departement_y'].isna()]
b["nom_commune_complet"].nunique()

2

### Supprimer les communes hors de la France Metropolitaine

A présent il y'a un problème sur St Barthelemy, et sur les communes françaises qui ne sont pas en France métropolitaine. 
Pour celles qui ne sont pas en France Métropolitaine, on va tout simplement les supprimer;

In [507]:
base_meteo_modifie_1 = base_meteo_modifie_1.dropna(subset=['nom_commune_complet'])
base_meteo_modifie_1 = base_meteo_modifie_1[~ (base_meteo_modifie_1['nom_commune_complet'] == 'St barthelemy')]
base_meteo_modifie_1 = base_meteo_modifie_1[~ (base_meteo_modifie_1['nom_commune_complet'] == 'St pierre')]

In [508]:
base_meteo_modifie_1.isna().sum()

num_station                              0
date_UTC                                 0
vitesse_vent                         22589
temperature                          14689
nebulosite_totale                  1143225
temperature_degre_celcius            14689
temperature_celcius_min_sur_24h    2108075
temperature_celcius_max_sur_24h    2108762
latitude_x                               0
longitude_x                              0
altitude                                 0
libelle_geolocalisation                  0
code_geolocalisation                242657
nom_departement_x                   242657
code_departement_x                  156963
nom_region_x                        242657
code_region_x                       156963
mois                                     0
visibilite_horizontale              489853
coordonnees                              0
nom_commune_x                            0
code_commune_INSEE                       0
nom_commune_postal                       0
code_postal

In [510]:
base_meteo_modifie_1.drop(['nom_departement_x', 'code_departement_x','nom_region_x', 'code_region_x','latitude_x', 'longitude_x','nom_commune_x'], axis=1, inplace=True)
base_meteo_modifie_1.rename(columns={'latitude_y': 'latitude', 'longitude_y':'longitude','nom_commune_y': 'nom_commune', 'code_departement_y' : 'code_departement', 'nom_departement_y': 'nom_departement', 'code_region_y' : 'code_region', 'nom_region_y':'nom_region'}, inplace=True)

## Agrégation des variables météorologiques

In [511]:
base_meteo_modifie_2=base_meteo_modifie_1

Il est ici question de construire les variables qui seront utilisées dans les analyses. Les données seront agrégées au niveau régional et à une fréquence journalière.

Il est ici question de construire les variables qui seront utilisées dans les analyses. Les données seront agrégées au niveau régional et à une fréquence journalière.

In [512]:
def extraire_sous_chaine(chaine, debut, fin):
    """
    Extrait la sous-chaîne de 'debut' à 'fin' (inclus) de la chaîne donnée.
    """
    if debut < 0 or fin >= len(chaine):
        raise ValueError("Indices de début ou de fin invalides.")
    return chaine[debut : fin + 1]

Nous appliquons la fonction à la base:

In [513]:
base_meteo_modifie_2['jour']=base_meteo_modifie_2['date_UTC'].apply(extraire_sous_chaine,debut=0,fin=9)
base_meteo_modifie_2['heure']=base_meteo_modifie_2['date_UTC'].apply(extraire_sous_chaine,debut=11,fin=18)

## Agrégation des variables à l'échelle départementale

Tout d'abord, agrégeons les variables 'temperature_degre_celcius','vitesse_vent', 'nebulosite_totale', 'humidite', 'visibilite_horizontale'

In [514]:
agregation_dep_mean = base_meteo_modifie_2.groupby(['jour','code_departement'], as_index=False).agg({'temperature_degre_celcius': 'mean', 'vitesse_vent': 'mean','nebulosite_totale':'mean','visibilite_horizontale':'mean'})
agregation_dep_mean = agregation_dep_mean.rename(columns={'jour':'jour','code_departement':'code_departement','temperature_degre_celcius':'temperature_degre_celcius_departement','vitesse_vent':'vitesse_vent_departement','nebulosite_totale':'nebulosite_totale_departement','visibilite_horizontale':'visibilite_horizontale_departement'})

In [515]:
base_meteo_modifie_3 = pd.merge(base_meteo_modifie_2, agregation_dep_mean, on=['jour','code_departement'], how='left')

In [516]:
dep.rename(columns={'INSEE_DEP': 'code_departement'}, inplace=True)
dep['code_departement'] = dep['code_departement'].astype(str)
base_meteo_modifie_4 = dep.merge(base_meteo_modifie_3, on='code_departement')
#base_meteo_modifie_4 = gpd.GeoDataFrame(pd.merge(base_meteo_modifie_3, data_geo, on=['code_departement'], how='left'),  geometry='geometry')

In [517]:
agregation_dep_mean.head()

,jour,code_departement,temperature_degre_celcius_departement,vitesse_vent_departement,nebulosite_totale_departement,visibilite_horizontale_departement
0,2010-01-01,10,0.750000,4.86250,100.000000,5250.000000
1,2010-01-01,12,2.825000,7.97500,95.000000,3000.000000
2,2010-01-01,13,15.144304,5.73662,56.964286,39766.666667
3,2010-01-01,14,1.400000,2.87500,95.000000,19125.000000
4,2010-01-01,15,25.962500,1.78750,NaN,NaN


### Aggrégation par région en appliquant les pondérations que sont les superficies des départements

Créons un dictionnaire pour stocker les critères d'agrégation

In [518]:
criteres_agregation = {'temperature_degre_celcius_departement': lambda x: np.average(x, weights=base_meteo_modifie_4.loc[x.index, 'superficie']),'vitesse_vent_departement': lambda x: np.average(x, weights=base_meteo_modifie_4.loc[x.index, 'superficie']),'nebulosite_totale_departement': lambda x: np.average(x, weights=base_meteo_modifie_4.loc[x.index, 'superficie']),'visibilite_horizontale_departement': lambda x: np.average(x, weights=base_meteo_modifie_4.loc[x.index, 'superficie'])}

Appliquons l'agrégation groupée en utilisant les critères spécifiés

In [519]:
agregation_reg_mean = base_meteo_modifie_4.groupby(['jour','code_region'], as_index=False).agg(criteres_agregation)

In [520]:
agregation_reg_mean.head()

,jour,code_region,temperature_degre_celcius_departement,vitesse_vent_departement,nebulosite_totale_departement,visibilite_horizontale_departement
0,2010-01-01,11.0,-0.062500,4.900000,85.714286,13437.500000
1,2010-01-01,24.0,1.424418,3.459109,96.569760,7092.446640
2,2010-01-01,27.0,2.237500,4.237500,89.285714,19625.000000
3,2010-01-01,28.0,2.283775,5.450085,74.496179,17019.038174
4,2010-01-01,32.0,0.238066,4.914637,46.586860,21470.137965


In [521]:
agregation_reg_mean = agregation_reg_mean.rename(columns={'jour':'jour','code_region':'code_region','temperature_degre_celcius_departement':'temperature_degre_celcius_region','vitesse_vent_departement':'vitesse_vent_region','nebulosite_totale_departement':'nebulosite_totale_region','visibilite_horizontale_departement':'visibilite_horizontale_region'})

Fusion de la base agrégée obtenue et de la base de départ dans une nouvelle base

In [522]:
base_meteo_modifie_5 = pd.merge(base_meteo_modifie_4, agregation_reg_mean, on=['jour','code_region'], how='left')

Aperçu de la base obtenue

In [523]:
base_meteo_modifie_5.columns

Index(['id', 'ID', 'NOM_M', 'NOM', 'code_departement', 'INSEE_REG', 'source',
       'territoire', 'geometry', 'superficie', 'num_station', 'date_UTC',
       'vitesse_vent', 'temperature', 'nebulosite_totale',
       'temperature_degre_celcius', 'temperature_celcius_min_sur_24h',
       'temperature_celcius_max_sur_24h', 'altitude',
       'libelle_geolocalisation', 'code_geolocalisation', 'mois',
       'visibilite_horizontale', 'coordonnees', 'code_commune_INSEE',
       'nom_commune_postal', 'code_postal', 'libelle_acheminement', 'ligne_5',
       'latitude', 'longitude', 'code_commune', 'article', 'nom_commune',
       'nom_commune_complet', 'nom_departement', 'code_region', 'nom_region',
       'jour', 'heure', 'temperature_degre_celcius_departement',
       'vitesse_vent_departement', 'nebulosite_totale_departement',
       'visibilite_horizontale_departement',
       'temperature_degre_celcius_region', 'vitesse_vent_region',
       'nebulosite_totale_region', 'visibilite_horizo

## Agrégation des autres variables (températures minimales et maximales)

Nous allons considérer que la température minimale/maximale observée avant 22h est la température minimale/maximale de la journée.

Crérons une base des températures minimales
On extrait le base de données pour chaque 22h

In [524]:
base_22h=base_meteo_modifie_2[base_meteo_modifie_2['heure']=='22:00:00']
agregation_reg_min_max=base_22h.groupby(['jour','code_region'], as_index=False).agg({'temperature_celcius_min_sur_24h': 'min', 'temperature_celcius_max_sur_24h': 'max'})
agregation_reg_min_max = agregation_reg_min_max.rename(columns={'jour':'jour','code_region':'code_region','temperature_celcius_min_sur_24h':'temperature_celcius_min_sur_24h_region','temperature_celcius_max_sur_24h':'temperature_celcius_max_sur_24h_region'})

Fusion de la base agrégée obtenue et de la base de départ dans une nouvelle base

In [564]:
base_meteo_modifie_6 = pd.merge(base_meteo_modifie_5, agregation_reg_min_max, on=['jour','code_region'], how='left')

Aperçu

In [565]:
base_meteo_modifie_6.head()

,id,ID,NOM_M,NOM,code_departement,INSEE_REG,source,territoire,geometry,superficie,...,temperature_degre_celcius_departement,vitesse_vent_departement,nebulosite_totale_departement,visibilite_horizontale_departement,temperature_degre_celcius_region,vitesse_vent_region,nebulosite_totale_region,visibilite_horizontale_region,temperature_celcius_min_sur_24h_region,temperature_celcius_max_sur_24h_region
0,DEPARTEM_FXX_00000000003,NaN,CORSE-DU-SUD,Corse-du-Sud,2A,94,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"MULTIPOLYGON (((8.59803 41.88006, 8.59803 41.8...",4.033539e+09,...,10.8500,2.1750,98.750,12857.142857,9.751286,1.682938,98.075943,13311.650146,NaN,NaN
1,DEPARTEM_FXX_00000000003,NaN,CORSE-DU-SUD,Corse-du-Sud,2A,94,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"MULTIPOLYGON (((8.59803 41.88006, 8.59803 41.8...",4.033539e+09,...,16.3125,3.2250,14.375,13000.000000,17.283143,2.523980,12.400975,13202.217226,NaN,NaN
2,DEPARTEM_FXX_00000000003,NaN,CORSE-DU-SUD,Corse-du-Sud,2A,94,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"MULTIPOLYGON (((8.59803 41.88006, 8.59803 41.8...",4.033539e+09,...,16.0875,2.3125,61.875,14750.000000,16.451491,2.103542,55.471455,13806.319613,NaN,NaN
3,DEPARTEM_FXX_00000000003,NaN,CORSE-DU-SUD,Corse-du-Sud,2A,94,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"MULTIPOLYGON (((8.59803 41.88006, 8.59803 41.8...",4.033539e+09,...,16.3000,2.1250,50.625,15000.000000,16.697694,2.219368,29.488485,19718.401935,NaN,NaN
4,DEPARTEM_FXX_00000000003,NaN,CORSE-DU-SUD,Corse-du-Sud,2A,94,IGN:EXPRESS-COG-CARTO-TERRITOIRE,metropole,"MULTIPOLYGON (((8.59803 41.88006, 8.59803 41.8...",4.033539e+09,...,13.7000,2.4375,81.250,14000.000000,13.275344,1.851070,87.316517,12045.233484,NaN,NaN


En tenant compte de cette nouvelle base de données, nous allons créer la variable Jour-ouvre (prenant la valeur 1 si le statut du jour est ouvré et 0 sinon) au sein de la base_meteo_modifie_6.

In [566]:
holiday_dataset['jour_ouvre'] = holiday_dataset['statut'].apply(lambda x: 1 if x == 'ouvré'else 0)
holiday_dataset.head()

,date,annee,jour,statut,jour_code,description,jour_et_date,jour_ouvre
0,2012-09-27,2012,jeudi,ouvré,4,None,jeudi 27,1
1,2012-10-01,2012,lundi,ouvré,1,None,lundi 1,1
2,2012-10-13,2012,samedi,week-end,6,None,samedi 13,0
3,2012-11-30,2012,vendredi,ouvré,5,None,vendredi 30,1
4,2012-12-25,2012,mardi,férié,2,Noël,mardi 25,0


## Fusion de la base_meteo_modifie_6 et la base_jour_ouvre_1

Suppression des colonnes dont on a pas besoin de la base_jour_ouvre. On renomme la colonne jour en Date pour pouvoir faire la fusion.

In [567]:
base_meteo_modifie_6 =base_meteo_modifie_6.rename(columns={'jour': 'date'})
colonnes_a_supprimer = ['annee', 'jour', 'jour_code', 'description', 'jour_et_date']
base_jour_ouvre_1 = holiday_dataset.drop(colonnes_a_supprimer, axis=1)
base_jour_ouvre_1 =base_jour_ouvre_1.rename(columns={'statut': 'Statut_du_jour'})

Fusion des bases sur la colonne 'date'

In [568]:
base_meteo_modifie_6.columns

Index(['id', 'ID', 'NOM_M', 'NOM', 'code_departement', 'INSEE_REG', 'source',
       'territoire', 'geometry', 'superficie', 'num_station', 'date_UTC',
       'vitesse_vent', 'temperature', 'nebulosite_totale',
       'temperature_degre_celcius', 'temperature_celcius_min_sur_24h',
       'temperature_celcius_max_sur_24h', 'altitude',
       'libelle_geolocalisation', 'code_geolocalisation', 'mois',
       'visibilite_horizontale', 'coordonnees', 'code_commune_INSEE',
       'nom_commune_postal', 'code_postal', 'libelle_acheminement', 'ligne_5',
       'latitude', 'longitude', 'code_commune', 'article', 'nom_commune',
       'nom_commune_complet', 'nom_departement', 'code_region', 'nom_region',
       'date', 'heure', 'temperature_degre_celcius_departement',
       'vitesse_vent_departement', 'nebulosite_totale_departement',
       'visibilite_horizontale_departement',
       'temperature_degre_celcius_region', 'vitesse_vent_region',
       'nebulosite_totale_region', 'visibilite_horizo

In [569]:
base_meteo_modifie_6 = pd.merge(base_meteo_modifie_6, base_jour_ouvre_1, on='date', how='inner')  # 'inner' spécifie une fusion interne

In [570]:
base_meteo_modifie_6.isna().sum()

id                                              0
ID                                        1804902
NOM_M                                           0
NOM                                             0
code_departement                                0
INSEE_REG                                       0
source                                          0
territoire                                      0
geometry                                        0
superficie                                      0
num_station                                     0
date_UTC                                        0
vitesse_vent                                13620
temperature                                 13743
nebulosite_totale                          840340
temperature_degre_celcius                   13743
temperature_celcius_min_sur_24h           1720958
temperature_celcius_max_sur_24h           1723144
altitude                                        0
libelle_geolocalisation                         0


In [542]:
data_weather_prep = base_meteo_modifie_6

In [543]:
data_weather_prep.columns

Index(['id', 'ID', 'NOM_M', 'NOM', 'code_departement', 'INSEE_REG', 'source',
       'territoire', 'geometry', 'superficie', 'num_station', 'date_UTC',
       'vitesse_vent', 'temperature', 'nebulosite_totale',
       'temperature_degre_celcius', 'temperature_celcius_min_sur_24h',
       'temperature_celcius_max_sur_24h', 'altitude',
       'libelle_geolocalisation', 'code_geolocalisation', 'mois',
       'visibilite_horizontale', 'coordonnees', 'code_commune_INSEE',
       'nom_commune_postal', 'code_postal', 'libelle_acheminement', 'ligne_5',
       'latitude', 'longitude', 'code_commune', 'article', 'nom_commune',
       'nom_commune_complet', 'nom_departement', 'code_region', 'nom_region',
       'jour', 'heure', 'temperature_degre_celcius_departement',
       'vitesse_vent_departement', 'nebulosite_totale_departement',
       'visibilite_horizontale_departement',
       'temperature_degre_celcius_region', 'vitesse_vent_region',
       'nebulosite_totale_region', 'visibilite_horizo

In [544]:
data_weather_prep['jour'] = pd.to_datetime(data_weather_prep['jour'])
data_weather_prep.set_index('jour', inplace=True)
data_weather_prep = data_weather_prep[data_weather_prep.index.year == annee_choisie]

Imputation des variables manquantes : KNN Imputer

On se dit que les zones ayant des données semblables météorologiques "se ressemblent"

On applique alors le KNN imputer

In [550]:
data_weather_prep.isna().sum()

id                                             0
ID                                        123898
NOM_M                                          0
NOM                                            0
code_departement                               0
INSEE_REG                                      0
source                                         0
territoire                                     0
geometry                                       0
superficie                                     0
num_station                                    0
date_UTC                                       0
vitesse_vent                                   0
temperature                                    0
nebulosite_totale                              0
temperature_degre_celcius                      0
temperature_celcius_min_sur_24h                0
temperature_celcius_max_sur_24h                0
altitude                                       0
libelle_geolocalisation                        0
code_geolocalisation

In [548]:
# Initialiser l'imputeur KNN
data_imputer = KNNImputer(n_neighbors=2)
data_imputer_agg = KNNImputer(n_neighbors=2)

#Colonne à imputer
imp_col = ["vitesse_vent", "temperature", "nebulosite_totale", "temperature_degre_celcius", "temperature_celcius_min_sur_24h", "temperature_celcius_max_sur_24h", "visibilite_horizontale"]
imp_col_agg = ["temperature_degre_celcius_departement", "vitesse_vent_departement", "nebulosite_totale_departement", "visibilite_horizontale_departement", "temperature_degre_celcius_region", "vitesse_vent_region", "nebulosite_totale_region", "visibilite_horizontale_region", "temperature_celcius_min_sur_24h_region", "temperature_celcius_max_sur_24h_region"] 
# Appliquer l'imputation
data_weather_prep[imp_col] = data_imputer.fit_transform(data_weather_prep[imp_col])
data_weather_prep[imp_col_agg] = data_imputer_agg.fit_transform(data_weather_prep[imp_col_agg])

/opt/mamba/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """propagate metadata from other to self"""
/opt/mamba/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """propagate metadata from other to self"""
